In [ ]:
import os
if not os.path.exists("tiny-imagenet-200/"):
    !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
    !unzip -q tiny-imagenet-200.zip
    print("Data downloaded!")
else:
    print("Data downloaded!")

In [1]:
import torch
import torchvision
import torch.nn as nn

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import PIL

import os
import random

from utils import helper_plot

%matplotlib inline

%config InlineBackend.figure_format = 'retina'

In [2]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

Running on the CPU


In [ ]:
# Constants
DATA_DIR = "tiny-imagenet-200"
VAL_DIR = "tiny-imagenet-200/val"
VAL_IMG_DIR = "tiny-imagenet-200/val/images"

In [ ]:
train_transforms = torchvision.transforms.Compose([
    torchvision.transforms.RandomRotation(30),
    torchvision.transforms.RandomResizedCrop(224),
    torchvision.transforms.RandomHorizontalFlip(),
    torchvision.transforms.RandomVerticalFlip(),
    torchvision.transforms.ToTensor()
])

test_transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(255), 
    torchvision.transforms.CenterCrop(224), 
    torchvision.transforms.ToTensor()
])

In [ ]:
train_dataset = torchvision.datasets.ImageFolder("tiny-imagenet-200/train", transform=train_transforms)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=128, shuffle=True, drop_last=True)
helper_plot.plot_grid(dataloader=train_dataloader)

In [ ]:
# Place corresponding images into respective folders
val_info = pd.read_csv("tiny-imagenet-200/val/val_annotations.txt", sep='\t', header=None, names=["File", "Class", "X", "Y", "H", "W"])
val_info.drop(["X", "Y", "H", "W"], axis=1, inplace=True)
for img, folder_name in zip(val_info["File"], val_info["Class"]):
    newpath = os.path.join(VAL_IMG_DIR, folder_name)
    if not os.path.exists(newpath):
        os.makedirs(newpath)
    if os.path.exists(os.path.join(VAL_IMG_DIR, img)):
        os.rename(os.path.join(VAL_IMG_DIR, img), os.path.join(newpath, img))

In [ ]:
val_dataset = torchvision.datasets.ImageFolder("tiny-imagenet-200/val/images", transform=test_transforms)
val_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, drop_last=True)
helper_plot.plot_grid(dataloader=val_dataloader)

In [ ]:
class AlexNet(nn.Module):
    def __init__(self, num_classes: int = 200, dropout: float = 0.5) -> None:
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(64, 192, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(192, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((6, 6))
        self.classifier = nn.Sequential(
            nn.Dropout(p=dropout),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(p=dropout),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [ ]:
model = AlexNet(num_classes=200)
model

In [ ]:
criterion = nn.CrossEntropyLoss()
optimiser = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9)

In [ ]:
from utils import misc

In [ ]:
for idx, (image, label) in enumerate(train_dataloader):
    print(idx, image.shape, label.shape)
    break

In [ ]:
epochs = 2
train_loss = 0
train_correct = 0
total = 0


for i in range(epochs):
    for index, (images, labels) in enumerate(train_dataloader):
        outputs = model.forward(image)
        loss = criterion(outputs, labels)
        loss.backward()
        optimiser.step()

        train_loss += loss.item()

        total += label.size(0)

        misc.progress_bar(index, len(train_dataloader), 'Loss: %.4f | Acc: %.3f%% (%d/%d)'
                         % (train_loss / (index + 1), 100. * train_correct / total, train_correct, total))

        optimiser.zero_grad()

        print(f'Epoch: [{i+1} / {epochs}], Step [{index + 1} / {len(train_dataloader)}], Loss: {loss.item()}')


print(train_loss, train_correct / total)

